In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
gc.enable()

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
s_Age = StandardScaler()
s_Room = StandardScaler()
s_Food = StandardScaler()
s_Shop = StandardScaler()
s_Spa = StandardScaler()
s_VR = StandardScaler()

train['Age'] = train['Age'].fillna(train.groupby('HomePlanet')['Age'].transform('median'))
train['s_Age'] = s_Age.fit_transform(np.array(train['Age']).reshape(-1,1))
train['s_Room'] = s_Room.fit_transform(np.array(train['RoomService']).reshape(-1,1))
train['s_Food'] = s_Food.fit_transform(np.array(train['FoodCourt']).reshape(-1,1))
train['s_Shop'] = s_Shop.fit_transform(np.array(train['ShoppingMall']).reshape(-1,1))
train['s_Spa'] = s_Spa.fit_transform(np.array(train['Spa']).reshape(-1,1))
train['s_VR'] = s_VR.fit_transform(np.array(train['VRDeck']).reshape(-1,1))

In [ ]:
train[['Passenger_GRP','Company']] = train['PassengerId'].str.split('_',expand=True)
train['Passenger_GRP'] = train['Passenger_GRP'].astype(int)
train['Company'] = train['Company'].astype(int)
train['Has_Company'] = train['Company'].map(lambda x:1 if x>1 else 0)
train[['deck','num','side']] = train['Cabin'].str.split('/',expand=True)

In [ ]:
train[['First Name','Last Name']] = train.Name.str.split(' ',expand=True)

In [ ]:
train['TotalSpend'] = train['RoomService']+train['FoodCourt']+train['ShoppingMall'] +train['Spa'] + train['VRDeck']

In [ ]:
train['PCTRoomService'] = train['RoomService'] / train['TotalSpend']
train['PCTFoodCourt'] = train['FoodCourt'] / train['TotalSpend']
train['PCTShoppingMall'] = train['ShoppingMall'] / train['TotalSpend']
train['PCTSpa'] = train['Spa'] / train['TotalSpend']
train['PCTVRDeck'] = train['VRDeck'] / train['TotalSpend']

In [ ]:
train[['VIP', 'CryoSleep']] = train[['VIP', 'CryoSleep']].astype(bool)
train['CryoSleep'] = train['CryoSleep'].fillna(True)
train['VIP'] = train['VIP'].fillna(True)

In [ ]:
train['ENC_RoomService'] = pd.qcut(train['RoomService'], 15,labels=False,duplicates='drop')
train['ENC_FoodCourt'] = pd.qcut(train['FoodCourt'], 30,labels=False,duplicates='drop')
train['ENC_ShoppingMall'] = pd.qcut(train['ShoppingMall'], 24,labels=False,duplicates='drop')
train['ENC_Spa'] = pd.qcut(train['Spa'], 24,labels=False,duplicates='drop')
train['ENC_VRDeck'] = pd.qcut(train['VRDeck'], 24,labels=False,duplicates='drop')
train['ENC_TotalSpend'] = pd.qcut(train['TotalSpend'],35,labels=False,duplicates='drop')
train['ENC_Age'] = pd.qcut(train['Age'],20,labels=False,duplicates='drop')
train['num'] = train['num'].fillna(-1)
train['num'] = train['num'].astype('int')
train['ENC_num'] = pd.qcut(train['num'],20,labels=False,duplicates='drop')

In [ ]:
train['deck'] = train.deck.fillna(-1)
deck_no = {'F':1,'G':2,'E':3,'B':4,'C':5,'D':6,'A':7,'T':8,-1:-1}
train['deck'] = train.deck.apply(lambda x:deck_no[x])

In [ ]:
#One hot encoder
from sklearn.preprocessing import OneHotEncoder
enc_HP = OneHotEncoder(handle_unknown='ignore',sparse=False)
HomePlanet = enc_HP.fit_transform(train.HomePlanet.values.reshape(-1,1))
HomePlanet = pd.DataFrame(HomePlanet,columns=['HP0','HP1','HP2','HP3'])
HomePlanet = HomePlanet.astype(int)

In [ ]:
enc_D = OneHotEncoder(handle_unknown='ignore',sparse=False)
Destination = enc_D.fit_transform(train.Destination.values.reshape(-1,1))
Destination = pd.DataFrame(Destination,columns=['D0','D1','D2','D3'])
Destination = Destination.astype(int)

In [ ]:
train = pd.concat([train,HomePlanet],axis=1)
train = pd.concat([train,Destination],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE_side = LabelEncoder()
LE_Last = LabelEncoder()
LE_home= LabelEncoder()
LE_dest= LabelEncoder()

LE_side.fit(train['side'])
LE_Last.fit(train['Last Name'])
LE_home.fit(train['HomePlanet'])
LE_dest.fit(train['Destination'])

train['side'] = LE_side.transform(train['side'])
train['Last Name'] = LE_Last.transform(train['Last Name'])
train['HomePlanet'] = LE_home.transform(train['HomePlanet'])
train['Destination'] = LE_dest.transform(train['Destination'])

y = train['Transported']
X = train.drop(['Transported','Name','Cabin','Passenger_GRP','First Name'],axis=1)


In [ ]:
X = X.apply(lambda x: x.fillna(x.mean()),axis=0)
X.isnull().sum()

In [ ]:
'''X['ENC_RoomService'] = X['ENC_RoomService'].astype(int)
X['ENC_FoodCourt'] = X['ENC_FoodCourt'].astype(int)
X['ENC_ShoppingMall'] = X['ENC_ShoppingMall'].astype(int)
X['ENC_Spa'] = X['ENC_Spa'].astype(int)
X['ENC_VRDeck'] = X['ENC_VRDeck'].astype(int)
X['ENC_Age'] = X['ENC_Age'].astype(int)'''

In [ ]:
'''from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
regr = RandomForestClassifier(max_depth=500, random_state=0)
scores = cross_validate(regr, X, y, cv=10,scoring=('accuracy'),return_train_score=False)
print(sum(scores['test_score'])/10)
regr.fit(X,y)'''

In [ ]:
'''from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(random_state=1,max_iter=1500)
scores = cross_validate(clf1, X, y, cv=10,scoring=('accuracy'),return_train_score=False)
print(sum(scores['test_score'])/10)'''

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate
clf = GradientBoostingClassifier(n_estimators=106, learning_rate=0.091,random_state=0)
#scores = cross_validate(clf, X, y, cv=10,scoring=('accuracy'),return_train_score=False)
#print(round(scores['test_score'].mean(),5),round(scores['test_score'].var(),5))

In [ ]:
clf.fit(X,y)
import shap
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X)

In [ ]:
shap.summary_plot(
    shap_values, X, feature_names=X.columns, plot_type="bar"
)

In [ ]:
X_imp = pd.DataFrame(list(zip(X.columns,abs(shap_values.mean(axis=0)))),columns=['Feature Name','Importance'])
X_imp = X_imp.sort_values('Importance',ascending=False)
X_imp = X_imp.reset_index(drop=True)
features = X_imp[X_imp['Importance']>0.0006]['Feature Name']

In [ ]:
clf = GradientBoostingClassifier(n_estimators=106, learning_rate=0.1,random_state=0)
#scores = cross_validate(clf, X[features], y, cv=10,scoring=('accuracy'),return_train_score=False)
#print(round(scores['test_score'].mean(),5),round(scores['test_score'].var(),5))

In [ ]:
clf.fit(X[features],y)
probs = clf.predict_proba(X[features]).T[0]
predict = [False if pred>=0.49 else True for pred in probs]

In [ ]:
Compare = pd.DataFrame(list(zip(predict,y)),columns = ['Pred','True'])
False_list = Compare[Compare['Pred']!=Compare['True']].index.tolist()

In [ ]:
import shap
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X[features])
shap.initjs()
shap.force_plot(y[False_list[0]],shap_values[False_list[0]],feature_names=features)

In [ ]:
shap.force_plot(y[2],shap_values[2],feature_names=features)

In [ ]:
shap.force_plot(y[3],shap_values[3],feature_names=features)

In [ ]:
shap.force_plot(y[False_list[8]],shap_values[False_list[8]],feature_names=features)

In [ ]:
shap.force_plot(y[False_list[2]],shap_values[False_list[2]],feature_names=features)

In [ ]:
#X[features].corr()

In [ ]:
'''from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('GR', clf), ('LR', clf1), ('RF', regr)],voting='soft')
scores = cross_validate(clf, X, y, cv=10,scoring=('accuracy'),return_train_score=False)
print(round(sum(scores['test_score'])/10,5))'''

In [ ]:
'''y_int = y.astype(int)
from catboost import CatBoostClassifier

cat = CatBoostClassifier(
    iterations=10, 
    learning_rate=0.091, 
    #loss_function='CrossEntropy'
)
scores = cross_validate(cat, X, y_int, cv=10,scoring=('accuracy'),return_train_score=False)
print(round(sum(scores['test_score'])/10,5))'''

In [ ]:
'''from sklearn import svm
svc = svm.SVC(kernel='rbf', C=.75,gamma='auto')
scores = cross_validate(svc, X, y, cv=2,scoring=('accuracy'),return_train_score=False)
print(sum(scores['test_score'])/2)'''

In [ ]:
'''from sklearn import svm
svc = svm.SVC(kernel='poly', C=5,degree=2)
scores = cross_validate(svc, X, y, cv=10,scoring=('accuracy'),return_train_score=False)
print(sum(scores['test_score'])/10)'''

In [ ]:
'''from sklearn.ensemble import ExtraTreesClassifier
extra_tree_forest = ExtraTreesClassifier(criterion ='entropy',)
scores = cross_validate(extra_tree_forest, X, y, cv=10,scoring=('accuracy'),return_train_score=False)
print(sum(scores['test_score'])/10)'''

In [ ]:
'''from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=75,metric='braycurtis')
scores = cross_validate(neigh, X, y, cv=10,scoring=('accuracy'),return_train_score=False)
print(sum(scores['test_score'])/10)'''

In [ ]:
test['Age'] = test['Age'].fillna(test.groupby('HomePlanet')['Age'].transform('median'))
test['s_Age'] = s_Age.fit_transform(np.array(test['Age']).reshape(-1,1))
test['s_Room'] = s_Room.fit_transform(np.array(test['RoomService']).reshape(-1,1))
test['s_Food'] = s_Food.fit_transform(np.array(test['FoodCourt']).reshape(-1,1))
test['s_Shop'] = s_Shop.fit_transform(np.array(test['ShoppingMall']).reshape(-1,1))
test['s_Spa'] = s_Spa.fit_transform(np.array(test['Spa']).reshape(-1,1))
test['s_VR'] = s_VR.fit_transform(np.array(test['VRDeck']).reshape(-1,1))

In [ ]:
test[['Passenger_GRP','Company']] = test['PassengerId'].str.split('_',expand=True)
test['Passenger_GRP'] = test['Passenger_GRP'].astype(int)
test['Company'] = test['Company'].astype(int)
test['Has_Company'] = test['Company'].map(lambda x:1 if x>1 else 0)
test[['deck','num','side']] = test['Cabin'].str.split('/',expand=True)

In [ ]:
test[['First Name','Last Name']] = test.Name.str.split(' ',expand=True)

In [ ]:
test['TotalSpend'] = test['RoomService']+test['FoodCourt']+test['ShoppingMall'] +test['Spa'] + test['VRDeck']

In [ ]:
test[['VIP', 'CryoSleep']] = test[['VIP', 'CryoSleep']].astype(bool)
test['CryoSleep'] = test['CryoSleep'].fillna(False)
test['VIP'] = test['VIP'].fillna(False)

In [ ]:
test['PCTRoomService'] = test['RoomService'] / test['TotalSpend']
test['PCTFoodCourt'] = test['FoodCourt'] / test['TotalSpend']
test['PCTShoppingMall'] = test['ShoppingMall'] / test['TotalSpend']
test['PCTSpa'] = test['Spa'] / test['TotalSpend']
test['PCTVRDeck'] = test['VRDeck'] / test['TotalSpend']

In [ ]:
test['ENC_RoomService'] = pd.qcut(test['RoomService'], 15,labels=False,duplicates='drop')
test['ENC_FoodCourt'] = pd.qcut(test['FoodCourt'], 30,labels=False,duplicates='drop')
test['ENC_ShoppingMall'] = pd.qcut(test['ShoppingMall'], 24,labels=False,duplicates='drop')
test['ENC_Spa'] = pd.qcut(test['Spa'], 24,labels=False,duplicates='drop')
test['ENC_VRDeck'] = pd.qcut(test['VRDeck'], 24,labels=False,duplicates='drop')
test['ENC_TotalSpend'] = pd.qcut(test['TotalSpend'],35,labels=False,duplicates='drop')
test['ENC_Age'] = pd.qcut(test['Age'],20,labels=False,duplicates='drop')
test['num'] = test['num'].fillna(-1)
test['num'] = test['num'].astype('int')
test['ENC_num'] = pd.qcut(test['num'],20,labels=False,duplicates='drop')

In [ ]:
test['deck'] = test.deck.fillna(-1)
deck_no = {'F':1,'G':2,'E':3,'B':4,'C':5,'D':6,'A':7,'T':8,-1:-1}
test['deck'] = test.deck.apply(lambda x:deck_no[x])

In [ ]:
#One hot encoder
HomePlanet = enc_HP.transform(test.HomePlanet.values.reshape(-1,1))
HomePlanet = pd.DataFrame(HomePlanet,columns=['HP0','HP1','HP2','HP3'])
HomePlanet = HomePlanet.astype(int)

In [ ]:
Destination = enc_D.transform(test.Destination.values.reshape(-1,1))
Destination = pd.DataFrame(Destination,columns=['D0','D1','D2','D3'])
Destination = Destination.astype(int)

In [ ]:
test = pd.concat([test,HomePlanet],axis=1)
test = pd.concat([test,Destination],axis=1)

In [ ]:
test['side'] = LE_side.transform(test['side'])

test['Last Name'] = test['Last Name'].map(lambda s: '<unknown>' if s not in LE_Last.classes_ else s)
LE_Last.classes_ = np.append(LE_Last.classes_, '<unknown>')
test['Last Name'] = LE_Last.transform(test['Last Name'])
test['HomePlanet'] = LE_home.transform(test['HomePlanet'])
test['Destination'] = LE_dest.transform(test['Destination'])


test_X = test.drop(['Name','Cabin','Passenger_GRP','First Name'],axis=1)


In [ ]:
test_X[test_X.columns[test_X.dtypes!='object']] = test_X[test_X.columns[test_X.dtypes!='object']].fillna(test_X[test_X.columns[test_X.dtypes!='object']].mean())
test_X.isnull().sum()

In [ ]:
'''test_X['ENC_RoomService'] =test_X['ENC_RoomService'].astype(int)
test_X['ENC_FoodCourt'] = test_X['ENC_FoodCourt'].astype(int)
test_X['ENC_ShoppingMall'] = test_X['ENC_ShoppingMall'].astype(int)
test_X['ENC_Spa'] = test_X['ENC_Spa'].astype(int)
test_X['ENC_VRDeck'] = test_X['ENC_VRDeck'].astype(int)
test_X['ENC_Age'] = test_X['ENC_Age'].astype(int)'''

In [ ]:
test_X.info()

In [ ]:
row_id = test['PassengerId']
#prediction = clf.predict(test_X)
#prediction = automl.predict(test_X[features])
probs = clf.predict_proba(test_X[features]).T[0]
prediction = [False if pred>=0.49 else True for pred in probs]

In [ ]:
submission = pd.DataFrame({'PassengerId':row_id,'Transported':prediction})
submission.to_csv('submission.csv',index=False)